## Test Conference
This conference is based on the test_iclr_conference_v2.py, with some simplifications

Some notes: 
- Some code cannot be run more than once without giving an error (for example if you withdraw a paper, then withdraw it again, it will give you an error)
- Most code depends on all of the code before it being run, since this is simulating a compressed conference workflow

In [42]:
from helpers import *
import time
import openreview
import pytest
import datetime
import re
import random
import os
import csv
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException
selenium = webdriver.Firefox()
helpers = Helpers()

## Make Sample SACs + ACs

In [43]:
#Set up deadlines
now = datetime.datetime.utcnow()
abstract_date = now + datetime.timedelta(days=1)
due_date = now + datetime.timedelta(days=3)

#connect to client
client = openreview.Client(baseurl = 'http://localhost:3000', username = 'openreview.net', password=helpers.strong_password)
openreview_client =openreview.Client(baseurl = 'http://localhost:3001', username = 'openreview.net', password=helpers.strong_password)

In [44]:

# Make SACs, ACs, Reviewers, and one test author
helpers.create_complete_user('sac10@mail.com', 'SAC', 'One')
sac_client = openreview.Client(username='sac10@mail.com', password=helpers.strong_password)

#Make ACs
helpers.create_complete_user('ac1@mail.com', 'AC', 'One')
ac_client = openreview.Client(username='ac1@mail.com', password=helpers.strong_password)

#Make Test User
helpers.create_complete_user('example@test.edu','SomeFirstName', 'User')
test_client = openreview.Client(username='example@test.edu', password=helpers.strong_password)



Look at a profile using this url: http://localhost:3030/profile?email=[EMAIL]

Example: http://localhost:3030/profile?email=pc@mail.com

All profiles use the same password: Or$3cur3P@ssw0rd (also stored in helpers.strong_password)

In [45]:
pc_client=openreview.Client(username='pc@mail.com', password=helpers.strong_password)
request_form=pc_client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]
venue = openreview.get_conference(client, request_form.id, support_user='openreview.net/Support')


In [46]:
helpers.strong_password

'Or$3cur3P@ssw0rd'

In [47]:
## Add SACs to SAC group, sample submission group
openreview_client = openreview.api.OpenReviewClient(baseurl = 'http://localhost:3001', username='openreview.net', password=Helpers.strong_password)

openreview_client.add_members_to_group('TC/2024/Conference/Senior_Area_Chairs', 
                                       [ '~SAC_One1'])


openreview_client.add_members_to_group('TC/2024/Conference/Submission1/Senior_Area_Chairs', 
                                       [ '~SAC_One1'])


Group(id = 'TC/2024/Conference/Submission1/Senior_Area_Chairs',invitation = None,invitations = ['TC/2024/Conference/Senior_Area_Chairs/-/Submission_Group', 'TC/2024/Conference/-/Edit'],content = None,cdate = 1734725830344,ddate = None,tcdate = 1734725830344,tmdate = 1734725870941,writers = ['TC/2024/Conference'],members = ['~SAC_One1'],readers = ['TC/2024/Conference', 'TC/2024/Conference/Program_Chairs', 'TC/2024/Conference/Submission1/Senior_Area_Chairs', 'TC/2024/Conference/Submission1/Area_Chairs', 'TC/2024/Conference/Submission1/Reviewers'],nonreaders = ['TC/2024/Conference/Submission1/Authors'],signatures = ['TC/2024/Conference'],signatories = ['TC/2024/Conference', 'TC/2024/Conference/Submission1/Senior_Area_Chairs'],anonids = None,web = None,impersonators = None,host = None,domain = 'TC/2024/Conference',parent = 'TC/2024/Conference/Submission1',deanonymizers = None,details = {'writable': True},anon_members = [])

In [48]:
#add ACs to AC group, sample submission group
openreview_client = openreview.api.OpenReviewClient(baseurl = 'http://localhost:3001', username='openreview.net', password=Helpers.strong_password)

openreview_client.add_members_to_group('TC/2024/Conference/Area_Chairs', 
                                       [ '~AC_One1'])


openreview_client.add_members_to_group('TC/2024/Conference/Submission1/Area_Chairs', 
                                       [ '~AC_One1'])

Group(id = 'TC/2024/Conference/Submission1/Area_Chairs',invitation = None,invitations = ['TC/2024/Conference/Area_Chairs/-/Submission_Group', 'TC/2024/Conference/-/Edit'],content = None,cdate = 1734725828230,ddate = None,tcdate = 1734725828230,tmdate = 1734725872495,writers = ['TC/2024/Conference'],members = ['~AC_One1'],readers = ['TC/2024/Conference', 'TC/2024/Conference/Program_Chairs', 'TC/2024/Conference/Submission1/Senior_Area_Chairs', 'TC/2024/Conference/Submission1/Area_Chairs', 'TC/2024/Conference/Submission1/Reviewers'],nonreaders = ['TC/2024/Conference/Submission1/Authors'],signatures = ['TC/2024/Conference'],signatories = ['TC/2024/Conference'],anonids = True,web = None,impersonators = None,host = None,domain = 'TC/2024/Conference',parent = 'TC/2024/Conference/Submission1',deanonymizers = ['TC/2024/Conference', 'TC/2024/Conference/Program_Chairs', 'TC/2024/Conference/Submission1/Senior_Area_Chairs', 'TC/2024/Conference/Submission1/Area_Chairs', 'TC/2024/Conference/Submissio

In [49]:
request_page(selenium, 'http://localhost:3030/group?id=TC/2024/Conference/Senior_Area_Chairs', sac_client.token, wait_for_element='header')

In [50]:
request_page(selenium, 'http://localhost:3030/group?id=TC/2024/Conference/Area_Chairs', ac_client.token, wait_for_element='header')

In [51]:
request_page(selenium, 'http://localhost:3030/group?id=TC/2024/Conference/Program_Chairs', pc_client.token, wait_for_element='header')